This notebook is part of https://github.com/AudioSceneDescriptionFormat/splines, see also https://splines.readthedocs.io/.

[back to overview](kochanek-bartels.ipynb)

# Non-Uniform Kochanek--Bartels Splines

<cite data-cite="kochanek1984tcb">Kochanek and Bartels (1984)</cite>
mainly talks about uniform splines.
Only in section 4, "Adjustments for Parameter Step Size", they briefly mention the non-uniform case.

TODO: show equations for adjusted tangents

Unfortunately, this is wrong.

TODO: show why it is wrong.

Instead, we should start from the correct tangent vector for non-uniform Catmull--Rom splines:

\begin{equation*}
\boldsymbol{\dot{x}}_i =
\frac{
(t_{i+1} - t_i)^2 (\boldsymbol{x}_i - \boldsymbol{x}_{i-1}) +
(t_i - t_{i-1})^2 (\boldsymbol{x}_{i+1} - \boldsymbol{x}_i)
}{
(t_{i+1} - t_i)(t_i - t_{i-1})(t_{i+1} - t_{i-1})
}
\end{equation*}

## Parameters

In general
incoming tangent $\boldsymbol{\dot{x}}_i^{(-)}$ and
outgoing tangent $\boldsymbol{\dot{x}}_i^{(+)}$ at vertex $\boldsymbol{x}_i$:

\begin{align*}
a_i &= (1 - T_i) (1 + C_i) (1 + B_i)\\
b_i &= (1 - T_i) (1 - C_i) (1 - B_i)\\
c_i &= (1 - T_i) (1 - C_i) (1 + B_i)\\
d_i &= (1 - T_i) (1 + C_i) (1 - B_i)
\end{align*}

\begin{align*}
\boldsymbol{\dot{x}}_i^{(+)} &= \frac{
a_i (t_{i+1} - t_i)^2 (\boldsymbol{x}_i - \boldsymbol{x}_{i-1}) +
b_i (t_i - t_{i-1})^2 (\boldsymbol{x}_{i+1} - \boldsymbol{x}_i)
}{(t_{i+1} - t_i) (t_i - t_{i-1}) (t_{i+1} - t_{i-1})}\\
\boldsymbol{\dot{x}}_i^{(-)} &= \frac{
c_i (t_{i+1} - t_i)^2 (\boldsymbol{x}_i - \boldsymbol{x}_{i-1}) +
d_i (t_i - t_{i-1})^2 (\boldsymbol{x}_{i+1} - \boldsymbol{x}_i)
}{(t_{i+1} - t_i) (t_i - t_{i-1}) (t_{i+1} - t_{i-1})}
\end{align*}

In the calculation below, we consider
the outgoing tangent at $\boldsymbol{x}_4$ and
the incoming tangent at $\boldsymbol{x}_5$.

\begin{align*}
a_4 &= (1 - T_4) (1 + C_4) (1 + B_4)\\
b_4 &= (1 - T_4) (1 - C_4) (1 - B_4)\\
c_5 &= (1 - T_5) (1 - C_5) (1 + B_5)\\
d_5 &= (1 - T_5) (1 + C_5) (1 - B_5)
\end{align*}

\begin{align*}
\boldsymbol{\dot{x}}_4^{(+)} &= \frac{
a_4 (t_5 - t_4)^2 (\boldsymbol{x}_4 - \boldsymbol{x}_3) +
b_4 (t_4 - t_3)^2 (\boldsymbol{x}_5 - \boldsymbol{x}_4)
}{(t_5 - t_4) (t_4 - t_3) (t_5 - t_3)}\\
\boldsymbol{\dot{x}}_5^{(-)} &= \frac{
c_5 (t_6 - t_5)^2 (\boldsymbol{x}_5 - \boldsymbol{x}_4) +
d_5 (t_5 - t_4)^2 (\boldsymbol{x}_6 - \boldsymbol{x}_5)
}{(t_6 - t_5) (t_5 - t_4) (t_6 - t_4)}
\end{align*}

## Calculation

In [ ]:
import sympy as sp
sp.init_printing()

In [ ]:
from utility import NamedExpression, NamedMatrix

In [ ]:
x3, x4, x5, x6 = sp.symbols('xbm3:7')

In [ ]:
t, t3, t4, t5, t6 = sp.symbols('t t3:7')

Same control values as Catmull-Rom ...

In [ ]:
control_values_KB = sp.Matrix([x3, x4, x5, x6])
control_values_KB

... but three additional parameters per vertex.
In our calculation, the parameters belonging to $\boldsymbol{x}_4$ and $\boldsymbol{x}_5$ are relevant:

In [ ]:
T4, T5 = sp.symbols('T4 T5')
C4, C5 = sp.symbols('C4 C5')
B4, B5 = sp.symbols('B4 B5')

In [ ]:
a4 = NamedExpression('a4', (1 - T4) * (1 + C4) * (1 + B4))
b4 = NamedExpression('b4', (1 - T4) * (1 - C4) * (1 - B4))
c5 = NamedExpression('c5', (1 - T5) * (1 - C5) * (1 + B5))
d5 = NamedExpression('d5', (1 - T5) * (1 + C5) * (1 - B5))
display(a4, b4, c5, d5)

In [ ]:
xd4 = NamedExpression(
    'xdotbm4^(+)',
    (a4.name * (t5 - t4)**2 * (x4 - x3) + b4.name * (t4 - t3)**2 * (x5 - x4)) /
    ((t5 - t4) * (t4 - t3) * (t5 - t3)))
xd5 = NamedExpression(
    'xdotbm5^(-)',
    (c5.name * (t6 - t5)**2 * (x5 - x4) + d5.name * (t5 - t4)**2 * (x6 - x5)) /
    ((t6 - t5) * (t5 - t4) * (t6 - t4)))
display(xd4, xd5)

In [ ]:
display(xd4.subs_symbols(a4, b4))
display(xd5.subs_symbols(c5, d5))

Same as with Catmull-Rom, try to find a transformation from cardinal control values to Hermite control values.
This can be used to get the full basis matrix.

In [ ]:
control_values_H = sp.Matrix([x4, x5, xd4.name, xd5.name])
control_values_H

From the [notebook about non-uniform Hermite splines](hermite-non-uniform.ipynb):

In [ ]:
M_H = NamedMatrix(
    r'{M_{\text{H},4}}',
    sp.Matrix([[2, -2, 1, 1],
               [-3, 3, -2, -1],
               [0, 0, 1, 0],
               [1, 0, 0, 0]]) * sp.diag(1, 1, t5 - t4, t5 - t4))
M_H

In [ ]:
M_KBtoH = NamedMatrix(r'{M_{\text{KB$,4\to$H},4}}', 4, 4)
M_KB = NamedMatrix(r'{M_{\text{KB},4}}', M_H.name * M_KBtoH.name)
M_KB

In [ ]:
NamedMatrix(control_values_H, M_KBtoH.name * control_values_KB)

If we substitute the above definitions of
$\boldsymbol{\dot{x}}_4^{(+)}$ and
$\boldsymbol{\dot{x}}_5^{(-)}$,
we can directly read off the matrix elements:

In [ ]:
M_KBtoH.expr = sp.Matrix([
    [expr.coeff(cv).simplify() for cv in control_values_KB]
    for expr in control_values_H.subs([xd4.args, xd5.args]).expand()])
M_KBtoH

In [ ]:
delta3, delta4, delta5 = sp.symbols('Delta3:6')
deltas = {
    t3: 0,
    t4: delta3,
    t5: delta3 + delta4,
    t6: delta3 + delta4 + delta5,
}

In [ ]:
M_KBtoH.subs(deltas).simplify()

In [ ]:
M_KB = M_KB.subs_symbols(M_H, M_KBtoH).doit()
M_KB.subs(deltas).expand()

And for completeness' sake, its inverse:

In [ ]:
M_KB.subs(deltas).expand().I

TODO: plot some example curves